In [8]:
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from dlc_practical_prologue import generate_pair_sets
from models import *
import copy

In [2]:
# Load data
train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(nb=1000)

# Prepare dataloader with class variable (Used to test auxiliary networks)
ds_train_class = torch.utils.data.TensorDataset(torch.cat((train_input[:, 0:1, :, :], train_input[:, 1:2, :, :]), 0), torch.cat((train_classes[:, 0], train_classes[:, 1]), 0))
ds_test_class = torch.utils.data.TensorDataset(torch.cat((test_input[:, 0:1, :, :], test_input[:, 1:2, :, :]), 0), torch.cat((test_classes[:, 0], test_classes[:, 1]), 0))
dl_train_class = torch.utils.data.DataLoader(ds_train_class, batch_size=32, shuffle=True, num_workers=4)
dl_test_class = torch.utils.data.DataLoader(ds_test_class, batch_size=32, shuffle=False, num_workers=4)

# Prepare dataloader with target variable (Used for networks without auxiliary loss)
ds_train_target = torch.utils.data.TensorDataset(train_input, train_target)
ds_test_target = torch.utils.data.TensorDataset(test_input, test_target)
dl_train_target = torch.utils.data.DataLoader(ds_train_target, batch_size=32, shuffle=True, num_workers=4)
dl_test_target = torch.utils.data.DataLoader(ds_test_target, batch_size=32, shuffle=False, num_workers=4)

# Prepare dataloader with target and class variable (Used for networks with auxiliary loss)
ds_train_all = torch.utils.data.TensorDataset(train_input, train_classes, train_target)
ds_test_all = torch.utils.data.TensorDataset(test_input, test_classes, test_target)
dl_train_all = torch.utils.data.DataLoader(ds_train_all, batch_size=32, shuffle=True, num_workers=4)
dl_test_all = torch.utils.data.DataLoader(ds_test_all, batch_size=32, shuffle=False, num_workers=4)

In [3]:
# Train and validate the Baseline Classifier (without auxiliary loss)

model = Baseline()
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_target, dl_test_target)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | conv1 | Conv2d           | 608   
2 | conv2 | Conv2d           | 18.5 K
3 | flat  | Flatten          | 0     
4 | fc1   | Linear           | 25.7 K
5 | fc2   | Linear           | 202   
-------------------------------------------
45.0 K    Trainable params
0         Non-trainable params
45.0 K    Total params
0.180     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:01<00:01, 20.58it/s, loss=2.7, v_num=45, val_loss=1.710, val_acc=0.578]
Validating: 0it [00:00, ?it/s]
Epoch 1:  62%|██████▎   | 40/64 [00:02<00:01, 17.57it/s, loss=0.557, v_num=45, val_loss=0.591, val_acc=0.686]
Validating: 0it [00:00, ?it/s]
Epoch 2:  62%|██████▎   | 40/64 [00:01<00:01, 20.16it/s, loss=0.415, v_num=45, val_loss=0.526, val_acc=0.728]
Validating: 0it [00:00, ?it/s]
Epoch 3:  62%|█████

1

In [7]:
# Train and validate the LeNet auxiliary network

lenet = LeNet()
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(lenet, dl_train_class, dl_test_class)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | conv1 | Conv2d           | 320   
2 | conv2 | Conv2d           | 18.5 K
3 | flat  | Flatten          | 0     
4 | fc1   | Linear           | 51.4 K
5 | fc2   | Linear           | 2.0 K 
-------------------------------------------
72.2 K    Trainable params
0         Non-trainable params
72.2 K    Total params
0.289     Total estimated model params size (MB)
Epoch 0:  63%|██████▎   | 80/126 [00:02<00:01, 27.47it/s, loss=0.56, v_num=49, val_loss=11.70, val_acc=0.000]
Validating: 0it [00:00, ?it/s]
Epoch 1:  63%|██████▎   | 80/126 [00:03<00:02, 20.11it/s, loss=0.31, v_num=49, val_loss=0.374, val_acc=0.882]
Validating: 0it [00:00, ?it/s]
Epoch 2:  63%|██████▎   | 80/126 [00:04<00:02, 19.81it/s, loss=0.268, v_num=49, val_loss=0.313, val_acc=0.901]
Validating: 0it [00:00, ?it/s]
Epoch 3:  63%|██

1

In [11]:
# Train and validate the LeNet classifier (with auxiliary loss)
auxiliary = LeNet()
model = CombinedNet(auxiliary)
trainer = pl.Trainer(
    max_epochs=25,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_all, dl_test_all)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | loss      | CrossEntropyLoss | 0     
1 | auxiliary | LeNet            | 72.2 K
2 | linear    | Linear           | 42    
-----------------------------------------------
72.3 K    Trainable params
0         Non-trainable params
72.3 K    Total params
0.289     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:02<00:01, 14.99it/s, loss=6.85, v_num=52, val_loss=5.090, val_acc=0.484]
Validating: 0it [00:00, ?it/s]
Epoch 1:  62%|██████▎   | 40/64 [00:02<00:01, 17.86it/s, loss=1.43, v_num=52, val_loss=0.654, val_acc=0.610]
Validating: 0it [00:00, ?it/s]
Epoch 2:  62%|██████▎   | 40/64 [00:02<00:01, 17.63it/s, loss=0.876, v_num=52, val_loss=0.530, val_acc=0.731]
Validating: 0it [00:00, ?it/s]
Epoch 3:  62%|██████▎   | 40/64 [00:02<00:01, 15.01it/s, loss=0.652, v_num=52, val_loss=0.444, val_acc=0.783]

1

In [10]:
# Train and validate the LeNet classifier with a pretrained lenet auxiliary network
model = CombinedNet(copy.deepcopy(lenet))
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_all, dl_test_all)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | loss      | CrossEntropyLoss | 0     
1 | auxiliary | LeNet            | 72.2 K
2 | linear    | Linear           | 42    
-----------------------------------------------
72.3 K    Trainable params
0         Non-trainable params
72.3 K    Total params
0.289     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:02<00:01, 17.05it/s, loss=1.27, v_num=51, val_loss=2.540, val_acc=0.438]
Validating: 0it [00:00, ?it/s]
Epoch 1:  62%|██████▎   | 40/64 [00:02<00:01, 15.22it/s, loss=0.486, v_num=51, val_loss=0.470, val_acc=0.795]
Validating: 0it [00:00, ?it/s]
Epoch 2:  62%|██████▎   | 40/64 [00:02<00:01, 13.85it/s, loss=0.296, v_num=51, val_loss=0.404, val_acc=0.823]
Validating: 0it [00:00, ?it/s]
Epoch 3:  62%|██████▎   | 40/64 [00:02<00:01, 13.74it/s, loss=0.25, v_num=51, val_loss=0.354, val_acc=0.849]

1